In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
home = pd.read_csv('home_completed.csv')
away = pd.read_csv('away_completed.csv')

In [3]:
#put all data into one large data frame
main = home.append(away, ignore_index=True).sort_values(['Checker']) # puts into date order
#put columns into more organized order
main = main[['Year','Month', 'Day' , 'Team', 'Spread', 'Line', '+/-', '3P', '3P%', '3PA', '3PAr', 'AST', 'BLK', 'DRB', 'Day',
       'FG', 'FG%', 'FGA', 'FT', 'FT%', 'FTA', 'FTr', 'Home', 'Losses',
        'ORB', 'PF', 'PTS', 'STL', 'Starters MP', 'TOV', 'TS%', 'Wins',
        'eFG%',  'Checker']]
#number of points a team won or lost by
main['+/-'] = main['+/-']/5
#puts the odds into floats
main['Spread'] = main['Spread'].replace(' PK', '0').astype(float)
#gets how many points a team went over or under the spread
main['Cover'] = main['Spread'] + main['+/-']
#true or false on whether a team covered the spread
main['ATS'] = np.where(main['Cover'].isnull(), np.nan,
          np.where(main['Cover'] > 0,   1, 0))
teams = list(main['Team'])
new_teams = []
#adds the opponent to the df
for i in range(0, len(teams), 2):
    new_teams.append(teams[i+1])
    new_teams.append(teams[i])
main['Opponent'] = new_teams
main = main.dropna()

In [4]:
def team_df(team):
    t = main[main['Team']== team].reset_index()
    del t['index']
    t['Season'] = t['Checker'].apply(season) #define what season a game is played
    t['Games Played'] = t['Wins'] + t['Losses'] #total games played
    #create running averages for all stats up to a given game date
    t_df = pd.DataFrame()
    for year in years:
        temp = t[t['Season']==year].reset_index()
        temp['Avg Win/Loss'] = (temp['+/-'].cumsum() - temp['+/-'])/temp.index
        temp['Avg 3P'] = (temp['3P'].cumsum() - temp['3P'])/temp.index
        temp['Avg 3PA'] = (temp['3PA'].cumsum() - temp['3PA'])/temp.index
        temp['Avg 3P%'] = (temp['Avg 3P']/temp['Avg 3PA'])
        temp['Avg Starter MP'] = (temp['Starters MP'].cumsum() - temp['Starters MP'])/temp.index
        temp['Avg AST'] = (temp['AST'].cumsum() - temp['AST'])/temp.index
        temp['Avg BLK'] = (temp['BLK'].cumsum() - temp['BLK'])/temp.index
        temp['Avg DRB'] = (temp['DRB'].cumsum() - temp['DRB'])/temp.index
        temp['Avg ORB'] = (temp['ORB'].cumsum() - temp['ORB'])/temp.index
        temp['Avg FG'] = (temp['FG'].cumsum() - temp['FG'])/temp.index
        temp['Avg FGA'] = (temp['FGA'].cumsum() - temp['FGA'])/temp.index
        temp['Avg FG%'] = (temp['Avg FG']/temp['Avg FGA'])
        temp['Avg FT'] = (temp['FT'].cumsum() - temp['FT'])/temp.index
        temp['Avg FTA'] = (temp['FTA'].cumsum() - temp['FTA'])/temp.index
        temp['Avg 3PAr'] = (temp['Avg 3PA']/temp['Avg FGA'])
        temp['Avg FTr'] = (temp['Avg FTA']/temp['Avg FGA'])
        temp['Avg 3P%'] = (temp['Avg FT']/temp['Avg FTA'])/temp.index
        temp['Avg PF'] = (temp['PF'].cumsum() - temp['PF'])/temp.index
        temp['Avg PTS'] = (temp['PTS'].cumsum() - temp['PTS'])/temp.index
        temp['Avg STL'] = (temp['STL'].cumsum() - temp['STL'])/temp.index
        temp['Avg TOV'] = (temp['TOV'].cumsum() - temp['TOV'])/temp.index
        temp['Avg TS%'] = temp['Avg PTS']/ (2*(temp['Avg FGA'] + .44 * temp['Avg FTA']))
        temp['Avg eFG%'] = (temp['Avg FG'] + .5 * temp['Avg 3P'])/ temp['Avg FGA']
        temp['Avg Starter MP'] = (temp['Starters MP'].cumsum() - temp['Starters MP'])/temp.index
        temp['Winning Percentage'] = temp['Wins']/temp.index
        t_df = t_df.append(temp)
    return t_df

In [5]:
def team_df(team):
    t = main[main['Team']== team].reset_index()
    del t['index']
    t['Season'] = t['Checker'].apply(season) #define what season a game is played
    t['Games Played'] = t['Wins'] + t['Losses'] #total games played
    #create running averages for all stats up to a given game date
    t_df = pd.DataFrame()
    for year in years:
        temp = t[t['Season']==year].reset_index()
        temp['Avg Win/Loss'] = (temp['+/-'].cumsum() - temp['+/-'])/temp.index
        temp['Avg 3P'] = (temp['3P'].cumsum() - temp['3P'])/temp.index
        temp['Avg 3PA'] = (temp['3PA'].cumsum() - temp['3PA'])/temp.index
        temp['Avg 3P%'] = (temp['Avg 3P']/temp['Avg 3PA'])
        temp['Avg Starter MP'] = (temp['Starters MP'].cumsum() - temp['Starters MP'])/temp.index
        temp['Avg AST'] = (temp['AST'].cumsum() - temp['AST'])/temp.index
        temp['Avg BLK'] = (temp['BLK'].cumsum() - temp['BLK'])/temp.index
        temp['Avg DRB'] = (temp['DRB'].cumsum() - temp['DRB'])/temp.index
        temp['Avg ORB'] = (temp['ORB'].cumsum() - temp['ORB'])/temp.index
        temp['Avg FG'] = (temp['FG'].cumsum() - temp['FG'])/temp.index
        temp['Avg FGA'] = (temp['FGA'].cumsum() - temp['FGA'])/temp.index
        temp['Avg FG%'] = (temp['Avg FG']/temp['Avg FGA'])
        temp['Avg FT'] = (temp['FT'].cumsum() - temp['FT'])/temp.index
        temp['Avg FTA'] = (temp['FTA'].cumsum() - temp['FTA'])/temp.index
        temp['Avg 3PAr'] = (temp['Avg 3PA']/temp['Avg FGA'])
        temp['Avg FTr'] = (temp['Avg FTA']/temp['Avg FGA'])
        temp['Avg 3P%'] = (temp['Avg FT']/temp['Avg FTA'])/temp.index
        temp['Avg PF'] = (temp['PF'].cumsum() - temp['PF'])/temp.index
        temp['Avg PTS'] = (temp['PTS'].cumsum() - temp['PTS'])/temp.index
        temp['Avg STL'] = (temp['STL'].cumsum() - temp['STL'])/temp.index
        temp['Avg TOV'] = (temp['TOV'].cumsum() - temp['TOV'])/temp.index
        temp['Avg TS%'] = temp['Avg PTS']/ (2*(temp['Avg FGA'] + .44 * temp['Avg FTA']))
        temp['Avg eFG%'] = (temp['Avg FG'] + .5 * temp['Avg 3P'])/ temp['Avg FGA']
        temp['Winning Percentage'] = temp['Wins']/temp.index
        ###
        temp['5 Game Win/Loss'] = temp['+/-'].rolling(5, min_periods=1).sum()-temp['+/-']
        temp['5 Game 3P'] = temp['3P'].rolling(5, min_periods=1).sum() - temp['3P']
        temp['5 Game 3PA'] = temp['3PA'].rolling(5, min_periods=1).sum()- temp['3PA']
        temp['5 Game 3P%'] = temp['5 Game 3P']/temp['5 Game 3PA']
        temp['5 Game Starter MP'] = temp['Starters MP'].rolling(5, min_periods=1).sum()-temp['Starters MP']
        temp['5 Game AST'] = temp['AST'].rolling(5, min_periods=1).sum() - temp['AST']
        temp['5 Game BLK'] = temp['BLK'].rolling(5, min_periods=1).sum() - temp['BLK']
        temp['5 Game DRB'] = temp['DRB'].rolling(5, min_periods=1).sum() - temp['DRB']
        temp['5 Game ORB'] = temp['ORB'].rolling(5, min_periods=1).sum() - temp['ORB']
        temp['5 Game FG'] = temp['FG'].rolling(5, min_periods=1).sum() - temp['FG']
        temp['5 Game FGA'] = temp['FGA'].rolling(5, min_periods=1).sum() - temp['FGA']
        temp['5 Game FG%'] = temp['5 Game FG']/temp['5 Game FGA']
        temp['5 Game FT'] = temp['FT'].rolling(5, min_periods=1).sum() - temp['FT']
        temp['5 Game FTA'] = temp['FTA'].rolling(5, min_periods=1).sum() - temp['FTA']
        temp['5 Game 3PAr'] = temp['5 Game 3PA']/temp['5 Game FGA']
        temp['5 Game FTr'] = temp['5 Game FTA']/temp['5 Game FGA']
        temp['5 Game 3P%'] = temp['5 Game FT']/temp['5 Game FTA']
        temp['5 Game PF'] = temp['PF'].rolling(5, min_periods=1).sum() - temp['PF']
        temp['5 Game PTS'] = temp['PTS'].rolling(5, min_periods=1).sum() - temp['PTS']
        temp['5 Game STL'] = temp['STL'].rolling(5, min_periods=1).sum() - temp['STL']
        temp['5 Game TOV'] = temp['TOV'].rolling(5, min_periods=1).sum() - temp['TOV']
        temp['5 Game TS%'] = temp['5 Game PTS']/ (2*(temp['5 Game FGA'] + .44 * temp['5 Game FTA']))
        temp['5 Game eFG%'] = (temp['5 Game FG'] + .5 * temp['5 Game 3P'])/ temp['5 Game FGA']
        t_df = t_df.append(temp)
    return t_df

In [6]:
#put each row into a season
def season(date):
    date = date[:6]
    if date < '200809':
        return '2007'
    elif date < '200909':
        return '2008'
    elif date < '201009':
        return '2009'
    elif date < '201109':
        return '2010'
    elif date < '201109':
        return '2010'
    elif date < '201209':
        return '2011'
    elif date < '201309':
        return '2012'
    elif date < '201409':
        return '2013'
    elif date < '201509':
        return '2014'
    elif date < '201609':
        return '2015'
    elif date < '201709':
        return '2016'
    elif date < '201809':
        return '2017'
    elif date < '201909':
        return '2018'
    

In [7]:
teams = set(teams)
years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
all_team_data = {}
for team in teams:
    all_team_data[team] = team_df(team)

In [8]:
giant_df = pd.DataFrame()
for df in all_team_data.values():
    giant_df = giant_df.append(df)

In [9]:
for team in teams:
    t = all_team_data[team]
    opp = giant_df[giant_df['Opponent']==team]
    opp = opp.rename(columns = {'Avg Win/Loss':'oppAvg Win/Loss', 'Avg 3P':'oppAvg 3P', 'Avg 3PA':'oppAvg 3PA' ,
       'Avg 3P%':'oppAvg 3P%', 'Avg Starter MP':'oppAvg Starter MP', 'Avg AST':'oppAvg AST', 'Avg BLK':'oppAvg BLK', 'Avg DRB':'oppAvg DRB', 'Avg ORB':'oppAvg ORB',
       'Avg FG':'oppAvg FG', 'Avg FGA':'oppAvg FGA', 'Avg FG%':'oppAvg FG%', 'Avg FT':'oppAvg FT', 'Avg FTA':'oppAvg FTA', 'Avg 3PAr':'oppAvg 3PAr',
       'Avg FTr':'oppAvg FTr', 'Avg PF':'oppAvg PF', 'Avg PTS':'oppAvg PTS', 'Avg STL':'oppAvg STL', 'Avg TOV':'oppAvg TOV', 'Avg TS%':'oppAvg TS%',
       'Avg eFG%':'oppAvg eFG%', 'Winning Percentage':'oppWinning Percentage'})
    opp = opp.rename(columns = {'5 Game Win/Loss':'opp 5 Game Win/Loss', '5 Game 3P':'opp 5 Game 3P', '5 Game 3PA':'opp 5 Game 3PA',
       '5 Game 3P%':'opp 5 Game 3P%', '5 Game Starter MP':'opp 5 Game Starter MP', '5 Game AST':'opp 5 Game AST', '5 Game BLK':'opp 5 Game BLK',
       '5 Game DRB':'opp 5 Game DRB', '5 Game ORB':'opp 5 Game ORB',
       '5 Game FG':'opp 5 Game FG', '5 Game FGA':'opp 5 Game FGA', '5 Game FG%':'opp 5 Game FG%', '5 Game FT':'opp 5 Game FT', '5 Game FTA':'opp 5 Game FTA', '5 Game 3PAr':'opp 5 Game 3PAr',
       '5 Game FTr':'opp 5 Game FTr', '5 Game PF':'opp 5 Game PF', '5 Game PTS':'opp 5 Game PTS', '5 Game STL':'opp 5 Game STL', '5 Game TOV':'opp 5 Game TOV', '5 Game TS%':'opp 5 Game TS%',
       '5 Game eFG%':'opp 5 Game eFG%'})
    opp = opp[['Checker', 'oppAvg Win/Loss', 'oppAvg 3P', 'oppAvg 3PA',
       'oppAvg 3P%', 'oppAvg Starter MP', 'oppAvg AST', 'oppAvg BLK',
       'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG', 'oppAvg FGA', 'oppAvg FG%',
       'oppAvg FT', 'oppAvg FTA', 'oppAvg 3PAr', 'oppAvg FTr', 'oppAvg PF',
       'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV', 'oppAvg TS%', 'oppAvg eFG%',
       'oppWinning Percentage', 'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game 3P%', 'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 'opp 5 Game FG', 'opp 5 Game FGA', 'opp 5 Game FG%',
       'opp 5 Game FT', 'opp 5 Game FTA', 'opp 5 Game 3PAr', 'opp 5 Game FTr', 'opp 5 Game PF',
       'opp 5 Game PTS', 'opp 5 Game STL', 'opp 5 Game TOV', 'opp 5 Game TS%', 'opp 5 Game eFG%']]
    all_team_data[team] = pd.merge(t,opp)

In [10]:
phi = all_team_data['Philadelphia']

In [11]:
total_matrix = phi[['Cover', 'ATS', '+/-', 'Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P',
       'Avg 3PA', 'Avg 3P%', 'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB',
       'Avg ORB', 'Avg FG', 'Avg FGA', 'Avg FG%', 'Avg FT', 'Avg FTA',
       'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL', 'Avg TOV',
       'Avg TS%', 'Avg eFG%', 'Winning Percentage', 'oppAvg Win/Loss','oppAvg 3P',
        'oppAvg 3PA', 'oppAvg 3P%', 'oppAvg Starter MP',
        'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG',
        'oppAvg FGA', 'oppAvg FG%', 'oppAvg FT', 'oppAvg FTA', 'oppAvg 3PAr',
        'oppAvg FTr', 'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV',
        'oppAvg TS%', 'oppAvg eFG%', 'oppWinning Percentage', '5 Game Win/Loss', '5 Game 3P', '5 Game 3PA',
       '5 Game 3P%', '5 Game Starter MP', '5 Game AST', '5 Game BLK',
       '5 Game DRB', '5 Game ORB', '5 Game FG', '5 Game FGA', '5 Game FG%',
       '5 Game FT', '5 Game FTA', '5 Game 3PAr', '5 Game FTr', '5 Game PF',
       '5 Game PTS', '5 Game STL', '5 Game TOV', '5 Game TS%', '5 Game eFG%',
       'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game 3P%', 'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 'opp 5 Game FG', 'opp 5 Game FGA', 'opp 5 Game FG%',
       'opp 5 Game FT', 'opp 5 Game FTA', 'opp 5 Game 3PAr', 'opp 5 Game FTr', 'opp 5 Game PF',
       'opp 5 Game PTS', 'opp 5 Game STL', 'opp 5 Game TOV', 'opp 5 Game TS%', 'opp 5 Game eFG%']]
total_matrix = total_matrix.dropna()


In [12]:
feature_matrix = total_matrix[['Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P',
       'Avg 3PA', 'Avg 3P%', 'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB',
       'Avg ORB', 'Avg FG', 'Avg FGA', 'Avg FG%', 'Avg FT', 'Avg FTA',
       'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL', 'Avg TOV',
       'Avg TS%', 'Avg eFG%', 'Winning Percentage']]
results = total_matrix['Cover']


In [13]:
feature_matrix = total_matrix[['Avg 3P%', 'Avg Starter MP', 'Avg AST', 'Avg ORB', 'Avg FG%', 'Avg FT', 'Avg FTA',
                               'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg TOV',
                               'Avg TS%', 'Avg eFG%', 'Winning Percentage', 'oppAvg Win/Loss','oppAvg 3P',
                               'oppAvg 3PA', 'oppAvg 3P%', 'oppAvg Starter MP',
                               'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG',
                               'oppAvg FGA', 'oppAvg FG%', 'oppAvg FT', 'oppAvg FTA', 'oppAvg 3PAr',
                               'oppAvg FTr', 'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV',
                               'oppAvg TS%', 'oppAvg eFG%', 'oppWinning Percentage']]

In [14]:
feature_matrix = total_matrix[['Avg ORB', 'Avg FTA',
                               'Avg 3PAr', 'Avg TOV',
                            'Avg eFG%', 'Winning Percentage', 
                               'oppAvg ORB', 'oppAvg FTA', 'oppAvg 3PAr',
                               'oppAvg TOV', 'oppAvg eFG%', 'oppWinning Percentage']]

In [15]:
feature_matrix = total_matrix[['Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P',
       'Avg 3PA', 'Avg 3P%', 'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB',
       'Avg ORB', 'Avg FG', 'Avg FGA', 'Avg FG%', 'Avg FT', 'Avg FTA',
       'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL', 'Avg TOV',
       'Avg TS%', 'Avg eFG%', 'Winning Percentage', 'oppAvg Win/Loss','oppAvg 3P',
        'oppAvg 3PA', 'oppAvg 3P%', 'oppAvg Starter MP',
        'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG',
        'oppAvg FGA', 'oppAvg FG%', 'oppAvg FT', 'oppAvg FTA', 'oppAvg 3PAr',
        'oppAvg FTr', 'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV',
        'oppAvg TS%', 'oppAvg eFG%', 'oppWinning Percentage', '5 Game Win/Loss', '5 Game 3P', '5 Game 3PA',
       '5 Game 3P%', '5 Game Starter MP', '5 Game AST', '5 Game BLK',
       '5 Game DRB', '5 Game ORB', '5 Game FG', '5 Game FGA', '5 Game FG%',
       '5 Game FT', '5 Game FTA', '5 Game 3PAr', '5 Game FTr', '5 Game PF',
       '5 Game PTS', '5 Game STL', '5 Game TOV', '5 Game TS%', '5 Game eFG%',
       'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game 3P%', 'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 'opp 5 Game FG', 'opp 5 Game FGA', 'opp 5 Game FG%',
       'opp 5 Game FT', 'opp 5 Game FTA', 'opp 5 Game 3PAr', 'opp 5 Game FTr', 'opp 5 Game PF',
       'opp 5 Game PTS', 'opp 5 Game STL', 'opp 5 Game TOV', 'opp 5 Game TS%', 'opp 5 Game eFG%']]

In [16]:
feature_matrix = total_matrix[['Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P',
       'Avg 3PA', 'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB',
       'Avg ORB', 'Avg FG', 'Avg FGA',
       'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL', 'Avg TOV',
       'Avg eFG%', 'oppAvg Win/Loss','oppAvg 3P',
        'oppAvg 3PA', 'oppAvg Starter MP',
        'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG',
        'oppAvg FGA', 'oppAvg 3PAr',
        'oppAvg FTr', 'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV',
        'oppAvg eFG%', 'oppWinning Percentage', '5 Game Win/Loss', '5 Game 3P', '5 Game 3PA',
       '5 Game Starter MP', '5 Game AST', '5 Game BLK',
       '5 Game DRB', '5 Game ORB', '5 Game FG', '5 Game FGA', 
       '5 Game 3PAr', '5 Game FTr', '5 Game PF',
       '5 Game PTS', '5 Game STL', '5 Game TOV', '5 Game eFG%',
       'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 'opp 5 Game FG', 'opp 5 Game FGA', 
       'opp 5 Game 3PAr', 'opp 5 Game FTr', 'opp 5 Game PF',
       'opp 5 Game PTS', 'opp 5 Game STL', 'opp 5 Game TOV', 'opp 5 Game eFG%']]

In [17]:
## model for against the spread
from sklearn import linear_model
from sklearn.metrics import make_scorer
model = linear_model.LinearRegression()
model.fit(feature_matrix.iloc[:712], results.iloc[:712])
predictions = model.predict(feature_matrix[712:])

In [18]:
def accuracy_predictor(lst1, lst2):
    lst = []
    for i in range(len(lst1)):
        x = (lst1[i]<0)
        y = (lst2[i]<0)
        if(lst1[i]==0 or lst2[i]==0):
            pass
        elif x == y:
            lst.append(1)
        else:
            lst.append(0)
    return lst

In [19]:
accuracy = accuracy_predictor(predictions, results.tolist()[712:])

In [20]:
sum(accuracy)/len(accuracy)

0.4700854700854701

In [21]:
x= pd.Series(model.coef_, feature_matrix.columns)
x.sort_values()

Avg eFG%                -1112.954802
oppAvg 3PAr              -349.722279
Avg 3PAr                 -261.033575
opp 5 Game eFG%          -128.372997
Avg FGA                    -8.937602
oppAvg FG                  -4.151804
opp 5 Game FTr             -3.731710
Avg BLK                    -2.982220
oppAvg 3P                  -1.778513
Home                       -1.489976
Avg TOV                    -1.287429
Avg Win/Loss               -0.844871
oppAvg STL                 -0.831790
oppAvg DRB                 -0.657945
Avg PTS                    -0.655080
Avg PF                     -0.527858
oppAvg BLK                 -0.505700
Wins                       -0.435333
Avg Starter MP             -0.392185
5 Game 3PA                 -0.320598
opp 5 Game 3PA             -0.243056
opp 5 Game FGA             -0.217415
oppAvg ORB                 -0.130666
5 Game TOV                 -0.128897
opp 5 Game AST             -0.120338
5 Game Starter MP          -0.113875
opp 5 Game Win/Loss        -0.106226
o

In [51]:
#model for straight victory
results = total_matrix['+/-']
results[results<0] = -1
results[results>0] = 1
model = linear_model.LinearRegression()
model.fit(feature_matrix.iloc[:712], results.iloc[:712])
predictions = model.predict(feature_matrix[712:])
accuracy = accuracy_predictor(predictions, results.tolist()[712:])
sum(accuracy)/len(accuracy)

/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


0.5084033613445378

In [23]:
# Import libraries for feature extraction
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


In [74]:
# Feature extraction
test = SelectKBest(score_func=f_classif, k=31)
fit = test.fit(feature_matrix.iloc[712:], results.iloc[712:])

# Summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)

features = fit.transform(feature_matrix)


[  4.679e+00   1.236e+00   1.013e+01   8.262e+00   6.938e-01   1.095e+00
   1.044e+01   7.144e+00   1.996e+00   5.024e+00   3.907e+00   6.137e+00
   8.988e-01   8.568e-01   6.100e+00   2.517e-01   6.351e+00   1.317e+00
   5.719e-01   7.291e+00   7.384e+00   1.673e-01   3.857e-02   5.064e-02
   8.571e-01   3.867e-01   4.671e-02   3.159e-01   4.073e-02   6.232e-01
   1.511e-03   1.152e-01   9.402e-03   2.502e-02   4.152e+00   2.964e-01
   1.237e+00   6.396e+00   8.908e+00   1.268e+00   3.019e-01   1.971e-02
   2.441e+00   3.217e-01   3.955e+00   1.619e-03   5.951e+00   3.530e+00
   1.151e+00   4.913e-02   3.947e-02   4.680e+00   2.366e+00   1.121e+00
   2.205e+00   8.067e+00   6.232e-02   9.812e-01   1.787e-01   6.165e-02
   4.639e-03   5.771e+00   4.835e-01   7.203e-01   4.680e+00   2.272e-02
   5.632e-04   1.814e+00   1.002e+00   1.776e+00   1.962e+00   4.048e+00]


In [75]:
# use just the 10 most important features
model = linear_model.LinearRegression()
model.fit(features[:712], results.iloc[:712])
predictions = model.predict(features[712:])
accuracy = accuracy_predictor(predictions, results.tolist()[712:])
sum(accuracy)/len(accuracy)

0.634453781512605

In [91]:
# Feature extraction ats
results = total_matrix['ATS']
results[results==0]=-1
test = SelectKBest(score_func=f_classif, k=20)
fit = test.fit(feature_matrix.iloc[712:], results.iloc[712:])

# Summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)

features = fit.transform(feature_matrix)


[  3.245e-02   6.691e-01   2.039e+00   1.728e+00   2.883e-01   1.642e+00
   9.177e-01   1.609e+00   2.381e+00   4.854e+00   2.060e-01   2.262e+00
   3.433e+00   4.399e-01   5.118e-01   1.576e-01   2.010e+00   2.386e-01
   2.190e+00   6.091e-01   2.601e-01   5.822e-02   2.625e-03   2.573e+00
   8.475e-02   1.097e+00   2.386e-02   7.716e-01   1.198e-02   3.248e-01
   9.829e-03   3.413e+00   2.607e-02   2.749e-01   3.656e-02   1.221e-01
   6.338e-02   2.705e-02   2.357e-01   1.095e+00   1.252e+00   5.748e-01
   1.578e-02   1.839e+00   6.148e-03   8.838e-02   9.015e-01   3.016e+00
   4.132e-02   1.158e+00   2.882e+00   3.482e-01   6.117e-01   9.101e-02
   2.410e-01   8.878e-01   1.463e-01   5.776e-01   2.330e+00   4.034e+00
   4.219e+00   5.023e+00   7.947e-01   3.030e+00   4.765e+00   1.420e-01
   1.790e+00   3.705e+00   4.659e+00   2.017e-01   2.701e+00   6.105e-02]


/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:
# use just the 10 most important features for betting ats
model = linear_model.LinearRegression()
model.fit(features[:712], results.iloc[:712])
predictions = model.predict(features[712:])
accuracy = accuracy_predictor(predictions, results.tolist()[712:])
sum(accuracy)/len(accuracy)

0.5

In [26]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

/Users/ericweltz/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ericweltz/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [211]:
results = total_matrix['+/-']
results[results>0]= 1
results[results<0]= -1

/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [212]:
xgbmat = xgb.DMatrix(feature_matrix.iloc[:712], results.iloc[:712])

In [217]:
cv_params = {'max_depth': [5,7, 9], 'min_child_weight': [5, 7, 9], 
             'learning_rate': [0.1, .01, 0.005, .001], 'n_estimators' : [50, 100, 500]}
ind_params = {'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 'objective':'reg:linear'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1)

In [218]:
optimized_GBM.fit(feature_matrix.iloc[:712], results.iloc[:712])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [5, 7, 9], 'min_child_weight': [5, 7, 9], 'learning_rate': [0.1, 0.01, 0.005, 0.001], 'n_estimators': [50, 100, 500]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [219]:
optimized_GBM.grid_scores_

[mean: 0.32584, std: 0.19456, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 50},
 mean: 0.31882, std: 0.19720, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 100},
 mean: 0.31180, std: 0.20230, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 500},
 mean: 0.33708, std: 0.19388, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 50},
 mean: 0.33287, std: 0.20208, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 100},
 mean: 0.32865, std: 0.20636, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 500},
 mean: 0.33006, std: 0.19266, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 9, 'n_estimators': 50},
 mean: 0.31742, std: 0.19737, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 9, 'n_estimators': 100},
 mean: 0.31742, std: 0.1961

In [220]:
results[results<0]=0
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':7, 'min_child_weight':5, 'n_estimators': 500} 
# Grid Search CV optimized settings

xgdmat = xgb.DMatrix(feature_matrix.iloc[:712], results.iloc[:712])

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 10,
                metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping

/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [221]:
cv_xgb

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.425352,0.077861,0.196557,0.022594
1,0.390141,0.078684,0.131456,0.014580
2,0.378873,0.058903,0.106729,0.012496
3,0.346479,0.051249,0.082785,0.015148
4,0.356338,0.048811,0.068545,0.009818
5,0.354930,0.049517,0.059937,0.007507
6,0.359155,0.059506,0.047105,0.009375
7,0.345070,0.059506,0.035211,0.007801
8,0.343662,0.072696,0.029734,0.007731
9,0.339437,0.061860,0.025039,0.005814


In [223]:
final_gb = xgb.train(our_params, xgdmat, num_boost_round = 14)

In [227]:
y = final_gb.predict(xgb.DMatrix(feature_matrix.iloc[:712]))
y = [round(i) for i in y]
z = results.iloc[:712]

In [228]:
def accuracy_predictor2(lst1, lst2):
    lst = []
    for i in range(len(lst1)):
        x = (lst1[i]<1)
        y = (lst2[i]<1)
        if(x==y):
            lst.append(1)
        else:
            lst.append(0)
    return lst

In [229]:
accuracy2 = accuracy_predictor2(y, z.tolist())
print(sum(accuracy2)/len(accuracy2))

0.9817415730337079


In [230]:
xgbmat = xgb.DMatrix(features[:712], results.iloc[:712])

In [231]:
cv_params = {'max_depth': [5,7, 9], 'min_child_weight': [5, 7, 9], 
             'learning_rate': [0.1, .01, 0.005, .001], 'n_estimators' : [50, 100, 500]}
ind_params = {'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 'objective':'reg:linear'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1)

In [233]:
optimized_GBM.fit(features[:712], results.iloc[:712])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [5, 7, 9], 'min_child_weight': [5, 7, 9], 'learning_rate': [0.1, 0.01, 0.005, 0.001], 'n_estimators': [50, 100, 500]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [234]:
optimized_GBM.grid_scores_

[mean: 0.32725, std: 0.18695, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 50},
 mean: 0.30337, std: 0.18775, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 100},
 mean: 0.31039, std: 0.18493, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 500},
 mean: 0.31039, std: 0.19287, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 50},
 mean: 0.31320, std: 0.18483, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 100},
 mean: 0.31180, std: 0.18229, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 500},
 mean: 0.31742, std: 0.18246, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 9, 'n_estimators': 50},
 mean: 0.30056, std: 0.18706, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 9, 'n_estimators': 100},
 mean: 0.30197, std: 0.1881

In [240]:
results[results<0]=0
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':7, 'min_child_weight':7, 'n_estimators': 500} 
# Grid Search CV optimized settings

xgdmat = xgb.DMatrix(features[:712], results.iloc[:712])

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 10,
                metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping

/Users/ericweltz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [241]:
cv_xgb

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.452113,0.080528,0.241784,0.013701
1,0.400000,0.077195,0.182316,0.011716
2,0.381690,0.058225,0.159311,0.013888
3,0.367606,0.060235,0.136776,0.012909
4,0.385915,0.057454,0.124413,0.011505
5,0.371831,0.052775,0.108607,0.008830
6,0.363380,0.073131,0.099217,0.006384
7,0.360564,0.057108,0.093114,0.009105
8,0.354930,0.069229,0.087637,0.008936
9,0.357747,0.064605,0.079186,0.007479


In [246]:
final_gb = xgb.train(our_params, xgdmat, num_boost_round = 10)
y = final_gb.predict(xgb.DMatrix(features[712:]))
y = [round(i) for i in y]
z = results.iloc[712:]
accuracy2 = accuracy_predictor2(y, z.tolist())
print(sum(accuracy2)/len(accuracy2))

0.5042016806722689


In [247]:
feature_matrix.shape

(950, 72)